In [1]:
sessionInfo()

R version 3.4.3 (2017-11-30)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 16.04.3 LTS

Matrix products: default
BLAS: /usr/lib/libblas/libblas.so.3.6.0
LAPACK: /usr/lib/lapack/liblapack.so.3.6.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] compiler_3.4.3      R6_2.2.2            magrittr_1.5       
 [4] IRdisplay_0.4.4     pbdZMQ_0.2-6        tools_3.4.3        
 [7] crayon_1.3.4        uuid_0.1-2          stringi_1.1.6      
[10] IRkernel_0.8.6.9000 jsonlite_1.5        stringr_1.2.0      
[13] digest_0.6.12     

In [2]:
library("DESeq2")

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colMeans, colnames,
    colSums, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, lengths, Map, mapply, match,
    mget, order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rowMeans, rownames, rowSums, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which, which.max, which.min


Attaching packag

In [3]:
setwd("/home/gstupp/projects/Wolan/ana_extractions/analysis/analysis_deseq")

In [4]:
file_path = "X.csv"
countData = read.csv(file_path, row.names = 1)

In [5]:
meta_path = "metadata.csv"
sampleTable = read.csv(meta_path, row.names = 1)
sampleTable = sampleTable[colnames(countData),]
sampleTable$cell = factor(sampleTable$cell)
sampleTable$method = factor(sampleTable$method)
sampleTable$method <- relevel(sampleTable$method, ref="sonic")
sampleTable = sampleTable[sampleTable$cell == "PA",]
sampleTable

,cell,file,method,name.1,pretty_name
PA_pellet_1,PA,data/PA/Pellet/PA pellet 1.txt,pellet,PA_pellet_1,PA pellet 1
PA_pellet_2,PA,data/PA/Pellet/PA pellet 2.txt,pellet,PA_pellet_2,PA pellet 2
PA_pellet_3,PA,data/PA/Pellet/PA pellet 3.txt,pellet,PA_pellet_3,PA pellet 3
PA_sonic_1,PA,data/PA/Sonic/PA sonic 1.txt,sonic,PA_sonic_1,PA sonic 1
PA_sonic_2,PA,data/PA/Sonic/PA sonic 2.txt,sonic,PA_sonic_2,PA sonic 2
PA_sonic_3,PA,data/PA/Sonic/PA sonic 3.txt,sonic,PA_sonic_3,PA sonic 3
PA_triflic_1,PA,data/PA/Triflic/PA triflic 1.txt,triflic,PA_triflic_1,PA triflic 1
PA_triflic_2,PA,data/PA/Triflic/PA triflic 2.txt,triflic,PA_triflic_2,PA triflic 2
PA_triflic_3,PA,data/PA/Triflic/PA triflic 3.txt,triflic,PA_triflic_3,PA triflic 3


## pellet vs sonic

In [6]:
sampleTableX = sampleTable[sampleTable$method %in% c("pellet", "sonic"),]
sampleTableX$method <- relevel(sampleTableX$method, ref="sonic")
X = countData[,sampleTableX$name]
X = X[ !apply(X, 1, function(x) all(is.na(x))), ]
X[is.na(X)] = 0
dds = DESeqDataSetFromMatrix(X, colData = sampleTableX, design = ~ method)
dds <- dds[ rowSums(counts(dds)) > 2, ]
counts(dds)[0:2,]

converting counts to integer mode
factor levels were dropped which had no samples


,PA_pellet_1,PA_pellet_2,PA_pellet_3,PA_sonic_1,PA_sonic_2,PA_sonic_3
E1JGJ8,3,0,4,7,2,0
G3XCT6,73,39,48,37,12,6


In [7]:
dds = DESeq(dds)
res = results(dds, alpha = .05)
summary(res)

estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing



out of 1844 with nonzero total read count
adjusted p-value < 0.05
LFC > 0 (up)     : 210, 11% 
LFC < 0 (down)   : 56, 3% 
outliers [1]     : 0, 0% 
low counts [2]   : 608, 33% 
(mean count < 2)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [8]:
res = results(dds)
resOrdered = res[order(res$padj),]
write.csv(as.data.frame(resOrdered),file="PA_deseq_results_pellet_sonic.csv")
resOrdered[0:10,]

log2 fold change (MLE): method pellet vs sonic 
Wald test p-value: method pellet vs sonic 
DataFrame with 10 rows and 6 columns
         baseMean log2FoldChange     lfcSE      stat       pvalue         padj
        <numeric>      <numeric> <numeric> <numeric>    <numeric>    <numeric>
Q9HWW1 1691.88383       7.060183 0.4113823 17.162095 5.103923e-66 6.675932e-63
P13794 1508.35968       8.743515 0.6594381 13.259039 3.999907e-40 2.615939e-37
Q9I4Z4  336.30225       6.600157 0.5071563 13.014049 1.018023e-38 4.438580e-36
Q9I4S1  256.00463       9.029736 0.8673229 10.411042 2.207914e-25 7.219880e-23
Q9HU11  173.26571       4.322857 0.4340763  9.958750 2.309478e-23 6.041595e-21
P32722  147.94640       7.480720 0.7708200  9.704886 2.873981e-22 6.265279e-20
O68822   93.47635       4.784102 0.5172053  9.249908 2.246850e-20 4.198401e-18
P00282  278.06677      -3.833872 0.4396448 -8.720385 2.772578e-18 4.533164e-16
G3XD89  133.24753       9.007335 1.1096176  8.117513 4.758337e-16 6.915450e-14
Q9H

## whole (triflic) vs sonic

In [9]:
sampleTableX = sampleTable[sampleTable$method %in% c("triflic", "sonic"),]
sampleTableX$method <- relevel(sampleTableX$method, ref="sonic")
X = countData[,sampleTableX$name]
X = X[ !apply(X, 1, function(x) all(is.na(x))), ]
X[is.na(X)] = 0
dds = DESeqDataSetFromMatrix(X, colData = sampleTableX, design = ~ method)
dds <- dds[ rowSums(counts(dds)) > 2, ]
counts(dds)[0:2,]

converting counts to integer mode
factor levels were dropped which had no samples


,PA_sonic_1,PA_sonic_2,PA_sonic_3,PA_triflic_1,PA_triflic_2,PA_triflic_3
E1JGJ8,7,2,0,0,0,0
G3XCT6,37,12,6,13,35,18


In [10]:
dds = DESeq(dds)
res = results(dds, alpha = .05)
summary(res)

estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing



out of 1850 with nonzero total read count
adjusted p-value < 0.05
LFC > 0 (up)     : 65, 3.5% 
LFC < 0 (down)   : 19, 1% 
outliers [1]     : 0, 0% 
low counts [2]   : 789, 43% 
(mean count < 3)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [11]:
res = results(dds)
resOrdered = res[order(res$padj),]
write.csv(as.data.frame(resOrdered),file="PA_deseq_results_triflic_sonic.csv")
resOrdered[0:10,]

log2 fold change (MLE): method triflic vs sonic 
Wald test p-value: method triflic vs sonic 
DataFrame with 10 rows and 6 columns
        baseMean log2FoldChange     lfcSE      stat       pvalue         padj
       <numeric>      <numeric> <numeric> <numeric>    <numeric>    <numeric>
Q9HWW1 556.09650       5.233479 0.5019778 10.425719 1.892232e-25 2.210127e-22
P13794 546.29212       7.079937 0.7299962  9.698594 3.056804e-22 1.785173e-19
Q9HVI2 301.44015      11.803715 1.3085240  9.020633 1.870061e-19 7.280771e-17
Q9HUM0 147.19566       7.388012 0.8299196  8.902081 5.480929e-19 1.600431e-16
Q9I4S1  49.57822       6.484473 0.9757671  6.645512 3.021639e-11 7.058549e-09
P32722  32.91916       5.144784 0.9510546  5.409557 6.318070e-08 1.229918e-05
G3XD89  28.13764       6.543555 1.2214481  5.357211 8.451630e-08 1.233938e-05
Q9I0X7  36.81547       6.147416 1.1458034  5.365158 8.087836e-08 1.233938e-05
Q9HVK6  19.52800       7.855018 1.5549107  5.051749 4.377832e-07 5.681453e-05
Q9HVH7  14.5

## Pellet vs. Whole cell (triflic)

In [12]:
sampleTableX = sampleTable[sampleTable$method %in% c("pellet", "triflic"),]
sampleTableX$method <- relevel(sampleTableX$method, ref="triflic")
X = countData[,sampleTableX$name]
X = X[ !apply(X, 1, function(x) all(is.na(x))), ]
X[is.na(X)] = 0
dds = DESeqDataSetFromMatrix(X, colData = sampleTableX, design = ~ method)
dds <- dds[ rowSums(counts(dds)) > 2, ]
counts(dds)[0:2,]

converting counts to integer mode
factor levels were dropped which had no samples


,PA_pellet_1,PA_pellet_2,PA_pellet_3,PA_triflic_1,PA_triflic_2,PA_triflic_3
E1JGJ8,3,0,4,0,0,0
G3XCT6,73,39,48,13,35,18


In [13]:
dds = DESeq(dds)
res = results(dds, alpha = .05)
summary(res)

estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing



out of 1829 with nonzero total read count
adjusted p-value < 0.05
LFC > 0 (up)     : 51, 2.8% 
LFC < 0 (down)   : 49, 2.7% 
outliers [1]     : 0, 0% 
low counts [2]   : 638, 35% 
(mean count < 2)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [14]:
res = results(dds)
resOrdered = res[order(res$padj),]
write.csv(as.data.frame(resOrdered),file="PA_deseq_results_pellet_triflic.csv")
resOrdered[0:10,]

log2 fold change (MLE): method pellet vs triflic 
Wald test p-value: method pellet vs triflic 
DataFrame with 10 rows and 6 columns
        baseMean log2FoldChange     lfcSE      stat       pvalue         padj
       <numeric>      <numeric> <numeric> <numeric>    <numeric>    <numeric>
Q9I4Z4 318.38724       3.297906 0.3464151  9.520100 1.730132e-21 1.998302e-18
Q9I4S1 264.01571       2.384003 0.3407880  6.995559 2.642031e-12 1.525773e-09
P30720 293.02556      -2.938590 0.4434354 -6.626873 3.428716e-11 1.320056e-08
P00282 101.52298      -2.434944 0.3740028 -6.510498 7.490215e-11 2.162800e-08
P32722 155.72917       2.204065 0.3560058  6.191094 5.974809e-10 1.380181e-07
G3XD89 139.16661       2.257997 0.3836707  5.885248 3.974585e-09 7.651077e-07
P72151 109.11990      -2.144049 0.3738051 -5.735739 9.708792e-09 1.601951e-06
Q9HVI2 286.73314      -3.330942 0.6020935 -5.532267 3.161187e-08 4.563963e-06
O68822  96.14928       2.157558 0.4101033  5.261012 1.432645e-07 1.838561e-05
P00106  58